In [30]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.util import ngrams

train_data_path = 'train.tsv'
test_data_path = 'valid.tsv'
output_path = 'output.txt'
stop_path = 'stopwords.txt'

In [31]:
train_df= pd.read_csv(train_data_path,header=None,sep='\t',quoting=3)
train_x= train_df[2]
train_y= train_df[1]
stop= pd.read_csv(stop_path,header=None)[0]
stop=stop.astype(str).tolist()

In [32]:
stemmer = PorterStemmer()

def preprocess_text(text):
    words = text.lower().split()
    # Filter out stopwords and apply stemming
    filtered_words = [stemmer.stem(word) for word in words if word not in stop]

    # Create unigrams (individual words)
    unigrams = filtered_words
    
    # Create bigrams (pairs of consecutive words)
    bigrams = ['_'.join(bigram) for bigram in ngrams(filtered_words, 2)]
    
    # Combine unigrams and bigrams
    return unigrams + bigrams

train_x= train_x.apply(preprocess_text)

In [33]:
label_to_index = {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}
index_to_label = {v: k for k, v in label_to_index.items()}


In [34]:
def create_feature_matrix(processed_texts, labels):
    vocabulary = set(word for text in processed_texts for word in text)
    vocab_size = len(vocabulary)
    
    word_to_index = {word: idx for idx, word in enumerate(vocabulary)}
    
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    class_count = np.zeros(len(set(labels)))

    # Fill the feature matrix
    for i, text in enumerate(processed_texts):
        for word in text:
            if word in word_to_index:
                feature_matrix[i, word_to_index[word]] += 1 

    # Encode labels
    encoded_labels = np.array([label_to_index[label] for label in labels])
    
    # Calculate class count as the sum of words in sentences of each class
    for i, label in enumerate(encoded_labels):
        class_count[label] += feature_matrix[i].sum()  # Sum of words in the sentence

    return feature_matrix, encoded_labels, class_count, word_to_index


X, Y, class_count, word_to_index= create_feature_matrix(train_x, train_y)

In [35]:
class MultinomialNaiveBayes:
    def __init__(self):
        self.class_priors = None
        self.feature_probs = None
        self.unseen_word_value_per_class = None

    def fit(self, X, y, class_count):
        self.class_priors = np.log(np.bincount(y) / len(y))
        self.feature_probs = np.zeros((len(self.class_priors), X.shape[1]))
        for c in range(len(self.class_priors)):
            X_c = X[y == c]
            self.feature_probs[c] = (X_c.sum(axis=0) + 1) / (X_c.sum() + X.shape[1])
        
        # class_count = np.bincount(y)
        self.unseen_word_value_per_class = [1 / (X.shape[1] + count) for count in class_count]

    def predict(self, X):
        eps = 1e-10 
        log_feature_probs = np.log(self.feature_probs.T.clip(eps, 1))
        log_probs = X @ log_feature_probs + self.class_priors
        print(log_probs)
        return np.argmax(log_probs, axis=1)

    def predict_test(self, X_test, unseen_word_count):
        eps = 1e-10 
        log_feature_probs = np.log(self.feature_probs.T.clip(eps, 1))
        log_probs = X_test @ log_feature_probs + self.class_priors
        for i in range(X_test.shape[0]): 
            for c in range(len(self.class_priors)): 
                if unseen_word_count[i] > 0:
                    unseen_contribution = unseen_word_count[i] * np.log(self.unseen_word_value_per_class[c])
                    log_probs[i, c] += unseen_contribution 
        
        print(log_probs)
        return np.argmax(log_probs, axis=1)


model = MultinomialNaiveBayes()
model.fit(X, Y, class_count)

predictions = model.predict(X)

accuracy = np.mean(predictions == Y)
print(f'Accuracy: {accuracy * 100:.2f}%')

[[-185.45151938 -172.99983292 -181.72029755 -181.94302542 -185.22959393
  -184.65105993]
 [-301.81531162 -291.80457439 -291.10050719 -281.73105164 -293.11483803
  -295.40259494]
 [-267.46008916 -256.48833336 -257.41741076 -257.73186898 -249.01183869
  -259.4139584 ]
 ...
 [-335.98118079 -329.41300567 -327.11515991 -313.94336048 -331.97032548
  -329.65756994]
 [-145.4951275  -137.72866481 -144.74182277 -145.60474022 -142.52171391
  -144.52220173]
 [-333.80902103 -347.65448754 -349.60287578 -349.97133151 -351.62657002
  -347.95865049]]
Accuracy: 96.08%


In [36]:
predicted_labels = [index_to_label[pred] for pred in predictions]
print(predicted_labels)
print(len(predicted_labels))
with open(output_path, 'w') as f:
    for label in predicted_labels:
        f.write(f"{label}\n")

['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'false', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'mostly-true', 'false', 'mostly-true', 'mostly-true', 'half-true', 'true', 'false', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'barely-true', 'false', 'mostly-true', 'mostly-true', 'true', 'true', 'true', 'pants-fire', 'true', 'false', 'half-true', 'pants-fire', 'pants-fire', 'false', 'half-true', 'pants-fire', 'false', 'pants-fire', 'true', 'barely-true', 'barely-true', 'false', 'true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'false', 'mostly-true', 'false', 'mostly-true', 'true', 'mostly-true', 'false', 'pants-fire', 'true', 'false', 'barely-true', 'barely-true', 'true', 'barely-true', 'mostly-true', 'mostly-true', 'mostly-true', 'true', 'false', 'barely-true', 'false', 'half-true', 'barely-true', 'half-true', 

In [37]:
file= 'checker_files/multinomial_bigrams_probas_train.npy'
given= np.load(file)
print(np.log(given))
argmax_indices = np.argmax(given, axis=1)
argmax_indices= [index_to_label[z] for z in argmax_indices]
print(argmax_indices)
print(len(argmax_indices))
print(argmax_indices==predicted_labels)
for i in range(10269):
    if argmax_indices[i]!=predicted_labels[i]:
        print(i)

[[-185.45151938 -172.99983292 -181.72029755 -181.94302542 -185.22959393
  -184.65105993]
 [-301.81531162 -291.80457439 -291.10050719 -281.73105164 -293.11483803
  -295.40259494]
 [-267.46008916 -256.48833336 -257.41741076 -257.73186898 -249.01183869
  -259.4139584 ]
 ...
 [-335.98118079 -329.41300567 -327.11515991 -313.94336048 -331.97032548
  -329.65756994]
 [-145.4951275  -137.72866481 -144.74182277 -145.60474022 -142.52171391
  -144.52220173]
 [-333.80902103 -347.65448754 -349.60287578 -349.97133151 -351.62657002
  -347.95865049]]
['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'false', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'mostly-true', 'false', 'mostly-true', 'mostly-true', 'half-true', 'true', 'false', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'barely-true', 'false', 'mostly-true', 'mostly-true', 'true', 'true', '

/var/folders/3x/n0vzxnz14dv3szr81wc5q8pm0000gn/T/ipykernel_20983/2609979564.py:3: RuntimeWarning: divide by zero encountered in log
  print(np.log(given))


In [38]:
test_df= pd.read_csv(test_data_path,header=None,sep='\t',quoting=3)
test_x= test_df[2]
test_y= test_df[1]

In [39]:
def create_test_feature_matrix(processed_texts, word_to_index):
    vocab_size = len(word_to_index)
    
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    unseen_word_count = np.zeros(len(processed_texts), dtype=int)
    
    for i, text in enumerate(processed_texts):
        for word in text:
            if word in word_to_index:
                feature_matrix[i, word_to_index[word]] += 1  # Count occurrences of each word
            else:
                unseen_word_count[i] += 1
    return feature_matrix, unseen_word_count

test_x= test_x.apply(preprocess_text)
X_test, unseen_word_count = create_test_feature_matrix(test_x, word_to_index)

test_predictions = model.predict_test(X_test, unseen_word_count)
test_predictions = [index_to_label[pred] for pred in test_predictions]
test_accuracy = np.mean(test_predictions == test_y)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


[[ -75.86619358  -73.31842188  -73.65828176  -70.64052947  -71.39921765
   -72.98468321]
 [-281.67801005 -281.31252867 -280.61344477 -281.13559429 -281.97594814
  -279.65123292]
 [-361.70494085 -354.7083752  -355.42133895 -355.22731076 -354.27695932
  -355.95721485]
 ...
 [-181.73186827 -174.75156193 -176.00982661 -171.92562389 -174.41863606
  -172.75120134]
 [-471.52541641 -458.02092118 -458.63159958 -456.63748365 -460.65626236
  -460.44962   ]
 [-252.10187553 -238.66348543 -244.23237104 -240.89214153 -242.29080784
  -244.4888989 ]]
Test Accuracy: 25.39%


In [40]:
print(test_predictions)

['half-true', 'true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'false', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'false', 'false', 'false', 'true', 'mostly-true', 'false', 'half-true', 'false', 'true', 'half-true', 'half-true', 'false', 'true', 'false', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'false', 'half-true', 'half-true', 'barely-true', 'barely-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'half-true', 'mostly-true', 'half-true', 'half-true', 'barely-true', 'barely-true', 'half-true', 'false', 'false', 'half-true', 'half-true', 'barely-true', 'true', 'mostly-true', 'false', 'mostly-true', 'barely-true', 'barely-true', 'barely-true', 'half-true', 'half-true', 'true', 'false', 'half-true', 'half-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'half-true', 'false', 'mostly-true', 'f

In [41]:
print(test_y)

0       barely-true
1        pants-fire
2             false
3         half-true
4         half-true
           ...     
1279      half-true
1280    mostly-true
1281           true
1282          false
1283    barely-true
Name: 1, Length: 1284, dtype: object


In [42]:
valfile= 'checker_files/multinomial_bigrams_probas_test.npy'
givenval= np.load(valfile)
print(np.log(givenval))
argmax_indices2 = np.argmax(givenval, axis=1)
argmax_indices2= [index_to_label[z] for z in argmax_indices2]
print(argmax_indices2)
print(len(argmax_indices2))
print(argmax_indices2==test_predictions)

[[ -75.86619358  -73.31842188  -73.65828176  -70.64052947  -71.39921765
   -72.98468321]
 [-281.67801005 -281.31252867 -280.61344477 -281.13559429 -281.97594814
  -279.65123292]
 [-361.70494085 -354.7083752  -355.42133895 -355.22731076 -354.27695932
  -355.95721485]
 ...
 [-181.73186827 -174.75156193 -176.00982661 -171.92562389 -174.41863606
  -172.75120134]
 [-471.52541641 -458.02092118 -458.63159958 -456.63748365 -460.65626236
  -460.44962   ]
 [-252.10187553 -238.66348543 -244.23237104 -240.89214153 -242.29080784
  -244.4888989 ]]
['half-true', 'true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'false', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'false', 'false', 'false', 'true', 'mostly-true', 'false', 'half-true', 'false', 'true', 'half-true', 'half-true', 'false', 'true', 'false', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'false

In [43]:
for i in range(1284):
    if argmax_indices2[i]!=test_predictions[i]:
        print(i)